#### Wczytanie bibliotek

In [6]:
import numpy as np
import pandas as pd
from datetime import datetime
import re

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.reset_option('display.max_columns')

#### Wczytanie danych

In [8]:
raw_data = pd.DataFrame()
for i in range(7):
    new_df = pd.read_csv(f"teams_pl_{23-i}-{24-i}_fbref.csv")
    raw_data = pd.concat([raw_data, new_df], ignore_index=True)
raw_data.head()

,season,venue_date,venue_time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,formation_home,formation_away,home_team,away_team,home_shirtnumber,home_nationality,home_position,home_age,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_shirtnumber,away_nationality,away_position,away_age,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_

#### Czyszczenie danych

In [9]:
raw_data.rename(columns={"venue_date": "date", "venue_time": "time"}, inplace=True)
raw_data = raw_data.drop(columns = ["home_shirtnumber", "home_nationality", "home_position", "home_age", "away_shirtnumber", "away_nationality", "away_position", "away_age"])

In [10]:
raw_data["home_goals"] = raw_data["home_goals"] + raw_data["away_own_goals"]
raw_data["away_goals"] = raw_data["away_goals"] + raw_data["home_own_goals"]
raw_data["home_goals_conceded"]=raw_data["away_goals"]
raw_data["away_goals_conceded"]=raw_data["home_goals"]

#### Dodanie informacji o punktach za mecz

In [11]:
def determine_outcome_and_points(row):
    if row['home_goals'] > row['away_goals']:
        return 1, 3, 0  # Home win
    elif row['home_goals'] < row['away_goals']:
        return 2, 0, 3  # Away win
    else:
        return 0, 1, 1  # Draw  
    
raw_data[['outcome', 'home_points', 'away_points']] = raw_data.apply(lambda row: pd.Series(determine_outcome_and_points(row)), axis=1)

#### Obliczenie statystyk z ostatnich meczów dla meczów domowych gospodarza i wyjazdowych gościa

In [12]:
def calculate_rolling_avg(period, column, new_column, home_away, df):
    df.sort_values(by=["season", home_away, "date"], inplace=True)
    df[new_column]=df.groupby(home_away)[column].transform(
        lambda x: x.shift(1).rolling(window=period, min_periods=0).mean()
    )   
    df[new_column].fillna(0.2, inplace=True)
    return df

In [13]:
columns_to_calculate_rolling_avg=raw_data.columns[15:]
home_columns_to_calculate_rolling_avg = columns_to_calculate_rolling_avg[columns_to_calculate_rolling_avg.str.startswith("home")]
away_columns_to_calculate_rolling_avg = columns_to_calculate_rolling_avg[columns_to_calculate_rolling_avg.str.startswith("away")]
for column in home_columns_to_calculate_rolling_avg:
    new_column_name = f"{column}_last5"
    raw_data = calculate_rolling_avg(5, column, new_column_name, "home_team", raw_data)

for column in away_columns_to_calculate_rolling_avg:
    new_column_name = f"{column}_last5"
    raw_data = calculate_rolling_avg(5, column, new_column_name, "away_team", raw_data)

raw_data = calculate_rolling_avg(10, "home_goals", "home_goals_last10", "home_team", raw_data)
raw_data = calculate_rolling_avg(10, "home_goals_conceded", "home_goals_conceded_last10", "home_team", raw_data)
raw_data = calculate_rolling_avg(20, "home_goals", "home_goals_last20", "home_team", raw_data)
raw_data = calculate_rolling_avg(20, "home_goals_conceded", "home_goals_conceded_last20", "home_team", raw_data)

raw_data = calculate_rolling_avg(10, "away_goals", "away_goals_last10", "away_team", raw_data)
raw_data = calculate_rolling_avg(10, "away_goals_conceded", "away_goals_conceded_last10", "away_team", raw_data)
raw_data = calculate_rolling_avg(20, "away_goals", "away_goals_last20", "away_team", raw_data)
raw_data = calculate_rolling_avg(20, "away_goals_conceded", "away_goals_conceded_last20", "away_team", raw_data)

C:\Users\Kuba\AppData\Local\Temp\ipykernel_23180\4274184508.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[new_column].fillna(0.2, inplace=True)
C:\Users\Kuba\AppData\Local\Temp\ipykernel_23180\4274184508.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when

In [17]:
raw_data[raw_data['home_goals_last5']==0]

,season,date,time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,home_goals_conceded,away_goals_conceded,outcome,home_points,away_points,home_goals_

In [15]:
raw_data.sort_values(["season", "date"], inplace=True)

In [10]:
raw_data[raw_data["home_team"]=="Arsenal"][["date", "round", "home_team", "home_goals", "home_goals_last5"]].head(30)

,date,round,home_team,home_goals,home_goals_last5
2280,2017-08-11,1,Arsenal,4,0.20
2312,2017-09-09,4,Arsenal,3,4.00
2339,2017-09-25,6,Arsenal,2,3.50
2347,2017-10-01,7,Arsenal,2,3.00
2371,2017-10-28,10,Arsenal,2,2.75
2390,2017-11-18,12,Arsenal,2,2.60
2416,2017-11-29,14,Arsenal,5,2.20
2427,2017-12-02,15,Arsenal,1,2.60
2452,2017-12-16,18,Arsenal,1,2.40
2460,2017-12-22,19,Arsenal,3,2.20


#### Dołącznie danych o stadionach

In [11]:
stadiums = pd.read_csv("stadiums.csv")
stadiums.head()

,Stadium,Club,Location,Opened,Closed,Capacity,Pitch length m,Pitch width m,latitude,longitude
0,Anfield,Liverpool,Liverpool,1884,NaN,61276,101.0,68.0,53°25′51″N,002°57′39″W
1,Arsenal Stadium(also known as Highbury),Arsenal,London,1913,2006.0,38419,NaN,NaN,51°33′28″N,000°06′10″W
2,Ayresome Park,Middlesbrough,Middlesbrough,1903,1995.0,26667,NaN,NaN,54°33′51″N,001°14′49″W
3,Baseball Ground,Derby County,Derby,1892,2004.0,18300,NaN,NaN,52°54′17″N,001°28′07″W
4,bet365 StadiumFormerly Britannia Stadium,Stoke City,Stoke-on-Trent,1997,NaN,30089,NaN,NaN,52°59′18″N,002°10′32″W


In [12]:
matches_with_stadiums = pd.merge(raw_data, stadiums[stadiums["Closed"].isna()], left_on=['home_team'], right_on=['Club'], how='left').drop(columns=["Closed", "Opened"])
matches_with_stadiums.head(2)

,season,date,time,round,attendance_value,referee,home_manager,away_manager,home_captain,away_captain,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,home_goals_conceded,away_goals_conceded,outcome,home_points,away_points,home_goals_

In [13]:
nan_columns = matches_with_stadiums.isna().sum()
print(nan_columns[nan_columns > 0])

attendance_value    441
home_manager        380
away_manager        380
home_captain        380
away_captain        380
Stadium             304
Club                304
Location            304
Capacity            304
Pitch length m      342
Pitch width m       342
latitude            304
longitude           304
dtype: int64


In [14]:
teams_without_stadium = matches_with_stadiums[matches_with_stadiums['Stadium'].isna()]['home_team'].unique()
teams_without_stadium

array(['West Bromwich Albion', 'Leicester City', 'Huddersfield Town',
       'Cardiff City', 'Norwich City', 'Leeds United'], dtype=object)

In [15]:
stadium_data = {
    'Leeds United': {
        'Stadium': 'Elland Road',
        'Location': 'Leeds',
        'Capacity': 37792,
        'Pitch length m': 105,
        'Pitch width m': 68,
        'Club': 'Leeds United',
        'latitude': 53.4640,
        'longitude': -1.3420
    },
    'Leicester City': {
        'Stadium': 'King Power Stadium',
        'Location': 'Leicester',
        'Capacity': 32261,
        'Pitch length m': 105,
        'Pitch width m': 68,
        'Club': 'Leicester City',
        'latitude': 52.6205,
        'longitude': -1.1425
    },
    'Norwich City': {
        'Stadium': 'Carrow Road',
        'Location': 'Norwich',
        'Capacity': 27300,
        'Pitch length m': 105,
        'Pitch width m': 67,
        'Club': 'Norwich City',
        'latitude': 52.3720,
        'longitude': 1.1833
    },
    'West Bromwich Albion': {
        'Stadium': 'The Hawthorns',
        'Location': 'West Bromwich',
        'Capacity': 26688,
        'Pitch length m': 105,
        'Pitch width m': 68,
        'Club': 'West Bromwich Albion',
        'latitude': 52.3033,
        'longitude': -1.5750
    },
    'Cardiff City': {
        'Stadium': 'Cardiff City Stadium',
        'Location': 'Cardiff',
        'Capacity': 33280,
        'Pitch length m': 100,
        'Pitch width m': 68,
        'Club' : 'Cardiff City',
        'latitude': 51.40703,
        'longitude': -3.2018
    },
    'Huddersfield Town': {
        'Stadium': 'John Smith\'s Stadium',
        'Location': 'Huddersfield',
        'Capacity': 24121,
        'Pitch length m': 105,
        'Pitch width m': 69,
        'Club': 'Huddersfield Town',
        'latitude': 53.6542,
        'longitude': -1.7692
    }
}


for team, stadium_info in stadium_data.items():
    for attribute, value in stadium_info.items():
        matches_with_stadiums.loc[matches_with_stadiums['home_team'] == team, attribute] = value

nan_columns = matches_with_stadiums.isna().sum()
print(nan_columns[nan_columns > 0])

attendance_value    441
home_manager        380
away_manager        380
home_captain        380
away_captain        380
Pitch length m       38
Pitch width m        38
dtype: int64


In [16]:
matches_with_stadiums = matches_with_stadiums.drop(columns = ["attendance_value", "home_manager", "away_manager", "home_captain", "away_captain"])
matches_with_stadiums['Pitch length m'] = matches_with_stadiums['Pitch length m'].fillna(105)
matches_with_stadiums['Pitch width m'] = matches_with_stadiums['Pitch width m'].fillna(68)

In [17]:
nan_columns = matches_with_stadiums.isna().sum()
print(nan_columns[nan_columns > 0])

Series([], dtype: int64)


#### Dodanie informacji o pogodzie

In [18]:
def degrees_to_decimal(geo_loc):
    if pd.isna(geo_loc):
        return 0.0
    if isinstance(geo_loc, float):
        return geo_loc
    geo_loc = str(geo_loc).strip()
    pattern = r'(\d+)°\s*(\d+)′\s*(\d+(\.\d+)?)″([NSEW])'
    match = re.match(pattern, geo_loc)
    if not match:
        raise ValueError(f"Invalid format: {geo_loc}")
    degrees = int(match.group(1))
    minutes = int(match.group(2))
    seconds = float(match.group(3))
    direction = match.group(5)
    decimal_loc = degrees + (minutes / 60) + (seconds / 3600)
    if direction in ['S', 'W']:
        decimal_loc = -decimal_loc
    return decimal_loc

In [19]:
matches_with_stadiums['longitude']=matches_with_stadiums['longitude'].apply(degrees_to_decimal)
matches_with_stadiums['latitude']=matches_with_stadiums['latitude'].apply(degrees_to_decimal)

In [20]:
from download_weather import get_weather

In [21]:
def create_weather_columns(row):
    data_weather=get_weather(
        latitude=row['latitude'],
        longitude=row['longitude'],
        start_hour=row['datetime'],
        end_hour=row['datetime'],
        params=['temperature_2m', 'precipitation', 'wind_speed_10m', 'relative_humidity_2m', 'cloud_cover'],
        type_url='historical'
    )
    row['weather_temperature']=data_weather['temperature_2m'][0]
    row['weather_precipitation']=data_weather['precipitation'][0]
    row['weather_wind']=data_weather['wind_speed_10m'][0]
    row['weather_humidity']=data_weather['relative_humidity_2m'][0]
    row['weather_cloud_cover']=data_weather['cloud_cover'][0]
    return row

In [22]:
matches_with_stadiums['datetime'] = pd.to_datetime(matches_with_stadiums['date'] + ' ' + matches_with_stadiums['time'])
matches_with_stadiums['datetime'] = matches_with_stadiums['datetime'].dt.strftime('%Y-%m-%dT%H:%M')
matches_with_stadiums.insert(1, 'datetime', matches_with_stadiums.pop('datetime'))
matches_with_stadiums.drop(columns=['date', 'time'], inplace=True)
matches_with_stadiums.head()

,season,datetime,round,referee,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,home_goals_conceded,away_goals_conceded,outcome,home_points,away_points,home_goals_last5,home_assists_last5,home_pens_made_last5,home_pens_att_last5,home

In [23]:
df= matches_with_stadiums.apply(create_weather_columns, axis=1)

In [105]:
matches_with_stadiums[:1]

,season,datetime,round,referee,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,home_goals_conceded,away_goals_conceded,outcome,home_points,away_points,home_goals_last5,home_assists_last5,home_pens_made_last5,home_pens_att_last5,home

In [24]:
df=df.sort_values(["season", "datetime", "round"])

In [25]:
df.head()

,season,datetime,round,referee,formation_home,formation_away,home_team,away_team,home_minutes,home_goals,home_assists,home_pens_made,home_pens_att,home_shots,home_shots_on_target,home_cards_yellow,home_cards_red,home_touches,home_tackles,home_interceptions,home_blocks,home_xg,home_npxg,home_xg_assist,home_sca,home_gca,home_passes_completed,home_passes,home_passes_pct,home_progressive_passes,home_carries,home_progressive_carries,home_take_ons,home_take_ons_won,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_pct_short,home_passes_completed_medium,home_passes_medium,home_passes_pct_medium,home_passes_completed_long,home_passes_long,home_passes_pct_long,home_pass_xa,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_crosses_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_through_balls,home_passes_switches,home_crosses,home_throw_ins,home_corner_kicks,home_corner_kicks_in,home_corner_kicks_out,home_corner_kicks_straight,home_passes_offsides,home_passes_blocked,home_tackles_won,home_tackles_def_3rd,home_tackles_mid_3rd,home_tackles_att_3rd,home_challenge_tackles,home_challenges,home_challenge_tackles_pct,home_challenges_lost,home_blocked_shots,home_blocked_passes,home_tackles_interceptions,home_clearances,home_errors,home_touches_def_pen_area,home_touches_def_3rd,home_touches_mid_3rd,home_touches_att_3rd,home_touches_att_pen_area,home_touches_live_ball,home_take_ons_won_pct,home_take_ons_tackled,home_take_ons_tackled_pct,home_carries_distance,home_carries_progressive_distance,home_carries_into_final_third,home_carries_into_penalty_area,home_miscontrols,home_dispossessed,home_passes_received,home_progressive_passes_received,home_cards_yellow_red,home_fouls,home_fouled,home_offsides,home_pens_won,home_pens_conceded,home_own_goals,home_ball_recoveries,home_aerials_won,home_aerials_lost,home_aerials_won_pct,away_minutes,away_goals,away_assists,away_pens_made,away_pens_att,away_shots,away_shots_on_target,away_cards_yellow,away_cards_red,away_touches,away_tackles,away_interceptions,away_blocks,away_xg,away_npxg,away_xg_assist,away_sca,away_gca,away_passes_completed,away_passes,away_passes_pct,away_progressive_passes,away_carries,away_progressive_carries,away_take_ons,away_take_ons_won,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_pct_short,away_passes_completed_medium,away_passes_medium,away_passes_pct_medium,away_passes_completed_long,away_passes_long,away_passes_pct_long,away_pass_xa,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_crosses_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_through_balls,away_passes_switches,away_crosses,away_throw_ins,away_corner_kicks,away_corner_kicks_in,away_corner_kicks_out,away_corner_kicks_straight,away_passes_offsides,away_passes_blocked,away_tackles_won,away_tackles_def_3rd,away_tackles_mid_3rd,away_tackles_att_3rd,away_challenge_tackles,away_challenges,away_challenge_tackles_pct,away_challenges_lost,away_blocked_shots,away_blocked_passes,away_tackles_interceptions,away_clearances,away_errors,away_touches_def_pen_area,away_touches_def_3rd,away_touches_mid_3rd,away_touches_att_3rd,away_touches_att_pen_area,away_touches_live_ball,away_take_ons_won_pct,away_take_ons_tackled,away_take_ons_tackled_pct,away_carries_distance,away_carries_progressive_distance,away_carries_into_final_third,away_carries_into_penalty_area,away_miscontrols,away_dispossessed,away_passes_received,away_progressive_passes_received,away_cards_yellow_red,away_fouls,away_fouled,away_offsides,away_pens_won,away_pens_conceded,away_own_goals,away_ball_recoveries,away_aerials_won,away_aerials_lost,away_aerials_won_pct,home_goals_conceded,away_goals_conceded,outcome,home_points,away_points,home_goals_last5,home_assists_last5,home_pens_made_last5,home_pens_att_last5,home

In [26]:
df.to_csv('pl_data_with_weather2.csv', index=False)

In [164]:
matches_with_stadiums.to_csv('pl_data_with_weather.csv', index=False)